In [24]:
from geopy.distance import distance
import pandas as pd
import numpy as np

## Bridge Elevations to Sidewalk Segments

In [25]:
bridge = pd.read_csv('elevation_road_bridge.csv')
elevations = pd.read_csv('elevations_metadata.csv')

In [26]:
elevations.head()

,OBJECTID,FID_Contours_Merge,Type,Elev,FID_tl_2015_55079_roads,LINEARID,Lat,Long
0,166,135516.0,Intermediate,783,886,1102213780402,42.979939,-88.037474
1,172,135522.0,Intermediate,779,886,1102213780402,42.979445,-88.037495
2,184,135574.0,Index,785,6446,110459338452,42.973745,-88.033920
3,185,135578.0,Intermediate,784,6446,110459338452,42.973900,-88.033929
4,186,135580.0,Intermediate,782,6446,110459338452,42.974104,-88.033918


In [27]:
bridge.head()

,FID_elevations_merged,FID_centerlines_dissolve
0,185,8136
1,187,8189
2,189,8136
3,191,8222
4,192,8136


In [28]:
bridge.set_index('FID_elevations_merged', drop=True, inplace=True)

In [29]:
bridge.head()

,FID_centerlines_dissolve
FID_elevations_merged,
185,8136
187,8189
189,8136
191,8222
192,8136


In [30]:
elevations.set_index('OBJECTID', drop=True, inplace=True)

In [31]:
elevations.shape

(151636, 7)

In [32]:
elevations = elevations.join(bridge,how='inner')

In [33]:
elevations.shape

(106867, 8)

In [34]:
elevations.reset_index(drop=False, inplace=True)

In [35]:
elevations.drop(['FID_Contours_Merge','Type','LINEARID','FID_tl_2015_55079_roads'], axis=1, inplace=True)
elevations.columns = ['elevation_objectID','elev','lat','long','roads_objectID']
elevations.head()

,elevation_objectID,elev,lat,long,roads_objectID
0,185,784,42.973900,-88.033929,8136
1,187,782,42.974420,-88.035083,8189
2,189,781,42.974227,-88.033921,8136
3,191,777,42.975043,-88.034076,8222
4,192,780,42.974342,-88.033920,8136


In [36]:
elevations.sort_values(by=['roads_objectID','lat','long'],inplace=True)
elevations.head()

,elevation_objectID,elev,lat,long,roads_objectID
23428,43937,765,42.920928,-87.939218,3371
23429,43953,766,42.921088,-87.939210,3371
23433,43964,767,42.921377,-87.939204,3371
2816,4374,759,42.921310,-87.937257,3379
2815,4370,760,42.921316,-87.937461,3379


In [16]:
elevations.to_csv('elevations_to_roads.csv')

## Find differences in Road Segments

In [52]:
df = pd.read_csv('elevations_to_roads.csv', index_col=0)
df.reset_index(drop=True, inplace=True)
df.head()

,elevation_objectID,elev,lat,long,roads_objectID
0,43937,765,42.920928,-87.939218,3371
1,43953,766,42.921088,-87.939210,3371
2,43964,767,42.921377,-87.939204,3371
3,4374,759,42.921310,-87.937257,3379
4,4370,760,42.921316,-87.937461,3379


In [53]:
def reset_grades(point):
    return point.elev, (point.lat, point.long), 0, 0

In [54]:
# Get all road segments and we'll loop through each one and put the 
# elevations in order
road_segments = np.unique(df.roads_objectID.values)
road_grades = {}
for road in road_segments:    
    elev = df[df.roads_objectID == road].copy()
    elev.drop_duplicates(subset=['elev','lat','long'], inplace=True)
    
    # If there's only one elevation, we can't capture road grade so 
    # set road grade to zero and move on
    if elev.shape[0] == 1:
        road_grades[road] = 0
        continue
        
    # Guess which direction the road is in based on max variance
    lat_diff = max(elev.lat.values) - min(elev.lat.values)
    long_diff = max(elev.long.values) - min(elev.long.values)
    
    # Sort values in order of whichever direction has max variance
    if lat_diff > long_diff:
        elev.sort_values(by='lat',inplace=True)
    else:
        elev.sort_values(by='long',inplace=True)
        
    # Calculate road grades per segment
    grades = []
    last_elev = elev.iloc[0].elev
    for ix, point in elev.iterrows():
        # For first point, just reset everything
        if ix == 0:
            last_elev, last_coord, diff, dist = reset_grades(point)
        else:
            # Calculate elevation change from most recent point
            this_diff = point.elev - last_elev 
            # If there's no elevation change, add the last grade and reset
            if this_diff == 0:
                # Add the last difference and reset
                if dist <= 5:
                    grades.append(0)
                else:
                    grades.append(abs(diff/dist))
                last_elev, last_coord, diff, dist = reset_grades(point)
            # If this is our second point, we don't care what direction we're moving in
            elif diff == 0:
                diff = this_diff
                dist = distance((point.lat,point.long),last_coord).ft
                last_coord = (point.lat,point.long)
            # If we're moving in the same direction, keep adding
            elif np.sign(diff) == np.sign(this_diff):
                diff += this_diff
                dist += distance((point.lat,point.long),last_coord).ft
                last_coord = (point.lat, point.long)
                last_elev = point.elev
            # If we're not moving in the same direction, add the last one and reset
                if dist <= 5:
                    grades.append(0)
                # Add the last one
                else:
                    grades.append(abs(diff/dist))
                # Reset
                diff = this_diff
                dist = distance((point.lat,point.long),last_coord).ft
                last_coord = (point.lat,point.long)
                last_elev = point.elev
    # Add in the last elevation
    if dist <= 5:
        grades.append(0)
    else:
        grades.append(abs(diff/dist))
        
    road_grades[road] = max(grades)

In [55]:
# with 10 ft min requirement
r = pd.Series(road_grades)
print(r[r>=0.1].shape[0],r[r>=0.05].shape[0],r[r<0.05].shape[0])

2866 5047 10646


In [56]:
r.to_csv('road_grades_by_segment.csv',header=['road_grade'], index_label = 'road_objectID')